# FINRA API Authentication Debug

This notebook tests FINRA API authentication to diagnose the 400 Bad Request error.

In [1]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from root .env
load_dotenv('.env')

FINRA_API_KEY = os.getenv('FINRA_API_KEY')
FINRA_API_SECRET = os.getenv('FINRA_API_SECRET')

print(f"API Key loaded: {FINRA_API_KEY[:10]}..." if FINRA_API_KEY else "API Key NOT FOUND")
print(f"API Secret loaded: {FINRA_API_SECRET[:5]}..." if FINRA_API_SECRET else "API Secret NOT FOUND")

API Key loaded: 3dd66476e1...
API Secret loaded: e9ZIg...


## Test 1: Token URL with grant_type in body only

FINRA OAuth2 typically expects `grant_type` in the request body, not the URL.

In [2]:
# Token URL WITHOUT grant_type in query string
TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

response = requests.post(
    TOKEN_URL,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    data={"grant_type": "client_credentials"},
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Headers: {dict(response.headers)}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

Status: 200
Headers: {'Date': 'Mon, 29 Dec 2025 20:56:33 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'cloudflare', 'Set-Cookie': 'AWSALB=MB4S0v6rDvf84oBhQYyvl6Y/sUbD29nva0wMqf3RPKELHcIZWhWeq3wrXOl7QgFel4U7JHOAesNUZih/Nb0oUb8+JkE+GSjPKKyVtGuyRgm6NbK17STvaVpcuUxQ; Expires=Mon, 05 Jan 2026 20:56:33 GMT; Path=/, AWSALBCORS=MB4S0v6rDvf84oBhQYyvl6Y/sUbD29nva0wMqf3RPKELHcIZWhWeq3wrXOl7QgFel4U7JHOAesNUZih/Nb0oUb8+JkE+GSjPKKyVtGuyRgm6NbK17STvaVpcuUxQ; Expires=Mon, 05 Jan 2026 20:56:33 GMT; Path=/; SameSite=None; Secure, AppSession=DBBC24BE-E4F8-11F0-9746-691216211806;Domain=.finra.org;Path=/;Secure;HttpOnly;Max-Age=315360000, AppSession=DBBC24BE-E4F8-11F0-9746-691216211806;Domain=.finra.org;Path=/;Secure;HttpOnly;Max-Age=315360000, __cf_bm=q71MEyQNATHzrzZ8Jshjt.79ypQtAyBkS6dZWDOAXSY-1767041793.7502396-1.0.1.1-BBh4o6FwTA81Nmd4tNwUizjXVBppacZWaSobKQdm3sDleMpdWuwQ6JW5PmZqwOx2l.9SgSFs2QyReNxWuq9vEKEfnjVCnapClPC5k_2GprEgVcjgq8_0MLAo

## Test 2: Token URL with grant_type in query string

Some APIs expect it in the URL instead.

In [3]:
# Token URL WITH grant_type in query string
TOKEN_URL_WITH_GRANT = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token?grant_type=client_credentials"

response = requests.post(
    TOKEN_URL_WITH_GRANT,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

Status: 200
Response: {"access_token":"*AAJTSQACMDIABHR5cGUAA0pXVAACUzEAAjAx*eyJ0eXAiOiJKV1QiLCJjdHkiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.ZXlKMGVYQWlPaUpLVjFRaUxDSmxibU1pT2lKQk1USTRRMEpETFVoVE1qVTJJaXdpWVd4bklqb2laR2x5SW4wLi5CNi1jSVNuM1hzRVVqbVNwLXcyaHhnLnFOZHlTY2lYMk1NcXlseGtqTUhCZF9nNHhCek1VNGVpWmdJLTNpbWxIVjNXcTdPdkpkRE1fUUNRaXFsdS1rd2FsbDVucmxDOF9Zd2NxWTNndldSU0J0WUJFemJTQVlORmxoeUlYTmMwWF9wYnIzU25ZZ3FuN3FhbzZDWENzT0ZVYlJxVjBLdVVCZWFfSlIwLWltaVg3U1BWTzh3b29NNDIzSndCb2xpelk5WVB0aDA2N1pZd0N0ZWVsblRtTVRQM2pzMVVKVkxFd3RyZVlR


## Test 3: Using headers instead of Basic Auth

Some FINRA endpoints use custom headers for authentication.

In [4]:
TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

# Try with X-API-KEY headers instead of Basic Auth
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "X-API-KEY": FINRA_API_KEY,
    "X-API-SECRET": FINRA_API_SECRET
}

response = requests.post(
    TOKEN_URL,
    headers=headers,
    data={"grant_type": "client_credentials"},
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

Status: 400
Response: {"error_message":"Client credentials is required","error":"invalid_client"}


## Test 4: Direct API call without token (API Key auth)

Some FINRA APIs allow direct API key authentication without OAuth token.

In [5]:
# Try direct API call with API key headers
OTC_URL = "https://api.finra.org/data/group/otcMarket/name/weeklySummary"

headers = {
    "Accept": "application/json",
    "X-API-KEY": FINRA_API_KEY
}

response = requests.get(
    OTC_URL,
    headers=headers,
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

Status: 200
Response: [{"totalWeeklyShareQuantity":6489206414,"issueSymbolIdentifier":null,"issueName":null,"lastUpdateDate":"2023-12-11","lastReportedDate":"2023-11-10","tierDescription":"Not Applicable","initialPublishedDate":"2023-12-11","tierIdentifier":"NMS","summaryStartDate":"2023-11-06","totalNotionalSum":347503970963,"totalWeeklyTradeCount":66391371,"weekStartDate":"2023-11-06","MPID":null,"firmCRDNumber":null,"productTypeCode":null,"marketParticipantName":null,"summaryTypeCode":"ATS_W_VOL_STATS"},{"totalWee


## Test 5: Check FINRA Gateway API directly

Try the FINRA Gateway API endpoint format.

In [6]:
# FINRA Gateway API format
GATEWAY_URL = "https://api.finra.org/data/group/otcMarket/name/weeklySummary"

# Using Basic Auth with the API credentials
response = requests.post(
    GATEWAY_URL,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    headers={"Accept": "application/json", "Content-Type": "application/json"},
    json={},  # Empty body for now
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:1000] if response.text else 'No response body'}")

Status: 200
Response: [{"totalWeeklyShareQuantity":6489206414,"issueSymbolIdentifier":null,"issueName":null,"lastUpdateDate":"2023-12-11","lastReportedDate":"2023-11-10","tierDescription":"Not Applicable","initialPublishedDate":"2023-12-11","tierIdentifier":"NMS","summaryStartDate":"2023-11-06","totalNotionalSum":347503970963,"totalWeeklyTradeCount":66391371,"weekStartDate":"2023-11-06","MPID":null,"firmCRDNumber":null,"productTypeCode":null,"marketParticipantName":null,"summaryTypeCode":"ATS_W_VOL_STATS"},{"totalWeeklyShareQuantity":21779,"issueSymbolIdentifier":"BRKR","issueName":"Bruker Corporation Common Stock","lastUpdateDate":"2023-11-27","lastReportedDate":"2023-11-10","tierDescription":"NMS Tier 1","initialPublishedDate":"2023-11-27","tierIdentifier":"NMS","summaryStartDate":"2023-11-06","totalNotionalSum":1257506,"totalWeeklyTradeCount":241,"weekStartDate":"2023-11-06","MPID":"JPBX","firmCRDNumber":null,"productTypeCode":null,"marketParticipantName":"JPBX JPB-X","summaryTypeCo

## Test 6: Correct FINRA CAT/OTC API format

Based on FINRA documentation, try the proper request format.

In [7]:
import base64

# Encode credentials for Authorization header
credentials = f"{FINRA_API_KEY}:{FINRA_API_SECRET}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

headers = {
    "Authorization": f"Basic {encoded_credentials}",
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(
    TOKEN_URL,
    headers=headers,
    data="grant_type=client_credentials",
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

if response.status_code == 200:
    token_data = response.json()
    print(f"\nAccess Token: {token_data.get('access_token', 'N/A')[:50]}...")

Status: 200
Response: {"access_token":"*AAJTSQACMDIABHR5cGUAA0pXVAACUzEAAjAx*eyJ0eXAiOiJKV1QiLCJjdHkiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.ZXlKMGVYQWlPaUpLVjFRaUxDSmxibU1pT2lKQk1USTRRMEpETFVoVE1qVTJJaXdpWVd4bklqb2laR2x5SW4wLi5GSGVJakpsN0pCWkdmSEt5eWowUW9nLnJueVhaM3R2bm5kamkwcWdPdG1HUHp3VGJkN1lGS2ttVF9HUEhBNGdPSm8ybUdOYzBVckNIX1JCS0duTGNTdmVBcUFBMTNwdVdqNjNGaXZZVmJUNUZPcHVmbFBQRzBVUVhCQURNLVN1Qm8tM18wRUo5c2t5bTFSRDVkXzVNcnpvWFBiVmxUNHJ2emFmeHZxY1Z4d0thMTIzTDNqaWhRa242R1YyMk11YmxLMzJPRF84czZ2a3M0cVdaTnJKbTFaUVdfZmhCRnFFYXdvM2xW

Access Token: *AAJTSQACMDIABHR5cGUAA0pXVAACUzEAAjAx*eyJ0eXAiOiJK...


## Summary

Based on the tests above, identify which authentication method works and update the code accordingly.

## DEBUG: Short Sale Daily API Investigation (2025-12-29)

The following cells investigate why FINRA Short Sale Daily data is returning empty responses.
Error: `Expecting value: line 1 column 1 (char 0)` indicates empty response body.

In [8]:
# Debug Cell 1: Fresh Token Generation Test
# Generate a new OAuth token and store it for subsequent tests

TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

token_response = requests.post(
    TOKEN_URL,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    data={"grant_type": "client_credentials"},
    timeout=30
)

print(f"Token Request Status: {token_response.status_code}")

if token_response.ok:
    token_data = token_response.json()
    ACCESS_TOKEN = token_data.get("access_token")
    print(f"Token obtained: Yes")
    print(f"Token prefix: {ACCESS_TOKEN[:50]}..." if ACCESS_TOKEN else "No token in response")
else:
    ACCESS_TOKEN = None
    print(f"Token generation FAILED!")
    print(f"Error: {token_response.text}")

Token Request Status: 200
Token obtained: Yes
Token prefix: *AAJTSQACMDIABHR5cGUAA0pXVAACUzEAAjAx*eyJ0eXAiOiJK...


In [9]:
# Debug Cell 2: Test Short Sale API Raw Response (regShoDaily)
# This tests the exact endpoint that's failing - inspect raw response BEFORE json parsing

SHORT_SALE_URL = "https://api.finra.org/data/group/otcMarket/name/regShoDaily"

headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

# Use a recent date - adjust if needed
payload = {
    "limit": 100,
    "dateRangeFilters": [
        {
            "fieldName": "tradeReportDate",
            "startDate": "2025-12-20",
            "endDate": "2025-12-20"
        }
    ]
}

print("Testing Short Sale API (regShoDaily)...")
print(f"URL: {SHORT_SALE_URL}")
print(f"Payload: {payload}")
print("-" * 50)

short_response = requests.post(
    SHORT_SALE_URL,
    headers=headers,
    json=payload,
    timeout=60
)

print(f"Status Code: {short_response.status_code}")
print(f"Content-Type: {short_response.headers.get('Content-Type', 'N/A')}")
print(f"Content-Length: {short_response.headers.get('Content-Length', 'N/A')}")
print(f"Response Body Length: {len(short_response.text)} chars")
print("-" * 50)
print(f"Raw Response (first 1000 chars):")
print(short_response.text[:1000] if short_response.text else "<<EMPTY RESPONSE>>")

Testing Short Sale API (regShoDaily)...
URL: https://api.finra.org/data/group/otcMarket/name/regShoDaily
Payload: {'limit': 100, 'dateRangeFilters': [{'fieldName': 'tradeReportDate', 'startDate': '2025-12-20', 'endDate': '2025-12-20'}]}
--------------------------------------------------
Status Code: 204
Content-Type: N/A
Content-Length: N/A
Response Body Length: 0 chars
--------------------------------------------------
Raw Response (first 1000 chars):
<<EMPTY RESPONSE>>


In [10]:
# Debug Cell 3: Test CDN Fallback (No Auth Required)
# The FINRA CDN is a public endpoint that doesn't require authentication
# This will tell us if the data exists at all

from datetime import date, timedelta

CDN_BASE = "https://cdn.finra.org/equity/regsho/daily"

print("Testing FINRA CDN (public, no auth required)...")
print("File format: CNMSshvol{YYYYMMDD}.txt")
print("-" * 50)

for days_ago in [1, 2, 3, 5, 7, 10]:
    test_date = date.today() - timedelta(days=days_ago)
    # Skip weekends
    if test_date.weekday() >= 5:
        continue
    
    filename = f"CNMSshvol{test_date.strftime('%Y%m%d')}.txt"
    url = f"{CDN_BASE}/{filename}"
    
    try:
        resp = requests.get(url, timeout=30)
        status = resp.status_code
        size = len(resp.text) if resp.ok else 0
        
        if resp.ok:
            # Show first few lines
            lines = resp.text.split('\n')[:3]
            preview = ' | '.join(lines)[:80]
            print(f"{test_date} ({test_date.strftime('%a')}): {status} OK - {size:,} bytes - {preview}...")
        else:
            print(f"{test_date} ({test_date.strftime('%a')}): {status} FAILED")
    except Exception as e:
        print(f"{test_date} ({test_date.strftime('%a')}): ERROR - {e}")

Testing FINRA CDN (public, no auth required)...
File format: CNMSshvol{YYYYMMDD}.txt
--------------------------------------------------
 | 20251226|A|18156...OK - 367,171 bytes - Date|Symbol|ShortVolume|ShortExemptVolume|TotalVolume|Market
 | 20251224|A|89573...OK - 358,292 bytes - Date|Symbol|ShortVolume|ShortExemptVolume|TotalVolume|Market
 | 20251222|A|93518...OK - 379,235 bytes - Date|Symbol|ShortVolume|ShortExemptVolume|TotalVolume|Market
 | 20251219|A|27337...OK - 374,801 bytes - Date|Symbol|ShortVolume|ShortExemptVolume|TotalVolume|Market


In [11]:
# Debug Cell 4: Test Multiple Date Ranges via API
# Check if the issue is date-specific

from datetime import date, timedelta

SHORT_SALE_URL = "https://api.finra.org/data/group/otcMarket/name/regShoDaily"
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

print("Testing Short Sale API across multiple dates...")
print("-" * 60)

test_offsets = [1, 3, 5, 7, 14, 30]  # Days ago

for days_ago in test_offsets:
    test_date = date.today() - timedelta(days=days_ago)
    
    # Skip weekends
    if test_date.weekday() >= 5:
        continue
    
    payload = {
        "limit": 10,
        "dateRangeFilters": [
            {
                "fieldName": "tradeReportDate",
                "startDate": test_date.isoformat(),
                "endDate": test_date.isoformat()
            }
        ]
    }
    
    try:
        resp = requests.post(SHORT_SALE_URL, headers=headers, json=payload, timeout=60)
        body_len = len(resp.text)
        
        # Try to parse as JSON to check structure
        try:
            data = resp.json()
            if isinstance(data, list):
                row_count = len(data)
            elif isinstance(data, dict) and "data" in data:
                row_count = len(data.get("data", []))
            else:
                row_count = "unknown structure"
        except:
            row_count = "JSON parse failed"
        
        print(f"{test_date} ({test_date.strftime('%a')}): Status={resp.status_code}, Body={body_len} chars, Rows={row_count}")
    except Exception as e:
        print(f"{test_date} ({test_date.strftime('%a')}): ERROR - {e}")

Testing Short Sale API across multiple dates...
------------------------------------------------------------
2025-12-26 (Fri): Status=204, Body=0 chars, Rows=JSON parse failed
2025-12-24 (Wed): Status=204, Body=0 chars, Rows=JSON parse failed
2025-12-22 (Mon): Status=204, Body=0 chars, Rows=JSON parse failed
2025-12-15 (Mon): Status=204, Body=0 chars, Rows=JSON parse failed


In [12]:
# Debug Cell 5: Compare OTC Weekly (Working) vs Short Sale Daily (Failing)
# Use the same token for both endpoints to isolate the issue

print("Comparison Test: OTC Weekly vs Short Sale Daily")
print("=" * 60)

# Same headers for both
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

# Test 1: OTC Weekly (known to work)
OTC_URL = "https://api.finra.org/data/group/otcMarket/name/weeklySummary"
otc_payload = {"limit": 5}

print("\n1. OTC Weekly Summary (weeklySummary):")
print(f"   URL: {OTC_URL}")
otc_resp = requests.post(OTC_URL, headers=headers, json=otc_payload, timeout=60)
print(f"   Status: {otc_resp.status_code}")
print(f"   Body length: {len(otc_resp.text)} chars")
try:
    otc_data = otc_resp.json()
    print(f"   JSON parsed: Yes ({type(otc_data).__name__})")
    if isinstance(otc_data, list):
        print(f"   Rows: {len(otc_data)}")
except Exception as e:
    print(f"   JSON parsed: FAILED - {e}")

# Test 2: Short Sale Daily (failing)
SHORT_SALE_URL = "https://api.finra.org/data/group/otcMarket/name/regShoDaily"
short_payload = {"limit": 5}

print("\n2. Short Sale Daily (regShoDaily) - NO date filter:")
print(f"   URL: {SHORT_SALE_URL}")
short_resp = requests.post(SHORT_SALE_URL, headers=headers, json=short_payload, timeout=60)
print(f"   Status: {short_resp.status_code}")
print(f"   Body length: {len(short_resp.text)} chars")
try:
    short_data = short_resp.json()
    print(f"   JSON parsed: Yes ({type(short_data).__name__})")
    if isinstance(short_data, list):
        print(f"   Rows: {len(short_data)}")
except Exception as e:
    print(f"   JSON parsed: FAILED - {e}")
    print(f"   Raw response: {short_resp.text[:200]}")

# Summary
print("\n" + "=" * 60)
print("DIAGNOSIS:")
if otc_resp.ok and len(otc_resp.text) > 10 and (not short_resp.ok or len(short_resp.text) < 10):
    print("  -> OTC works but Short Sale fails")
    print("  -> Issue is specific to regShoDaily endpoint")
    print("  -> Possible causes: API endpoint down, permission revoked, or endpoint changed")
    print("  -> RECOMMENDATION: Use CDN fallback (cdn.finra.org) as workaround")
elif not otc_resp.ok and not short_resp.ok:
    print("  -> Both endpoints failing")
    print("  -> Likely authentication or general API issue")
else:
    print("  -> Both endpoints working - issue may be date-specific")

Comparison Test: OTC Weekly vs Short Sale Daily

1. OTC Weekly Summary (weeklySummary):
   URL: https://api.finra.org/data/group/otcMarket/name/weeklySummary
   Status: 200
   Body length: 2560 chars
   JSON parsed: Yes (list)
   Rows: 5

2. Short Sale Daily (regShoDaily) - NO date filter:
   URL: https://api.finra.org/data/group/otcMarket/name/regShoDaily
   Status: 204
   Body length: 0 chars
   JSON parsed: FAILED - Expecting value: line 1 column 1 (char 0)
   Raw response: 

DIAGNOSIS:
  -> OTC works but Short Sale fails
  -> Issue is specific to regShoDaily endpoint
  -> Possible causes: API endpoint down, permission revoked, or endpoint changed
  -> RECOMMENDATION: Use CDN fallback (cdn.finra.org) as workaround


## Next Steps Based on Results

**If CDN works but API fails:**
- Modify `fetch_finra_short.py` to use CDN as primary source
- CDN endpoint: `https://cdn.finra.org/equity/regsho/daily/CNMSshvol{YYYYMMDD}.txt`

**If both fail:**
- Check FINRA API status: https://gateway.finra.org/app/api-console
- Regenerate API credentials
- Check if API access has expired

**If API returns empty for specific dates:**
- FINRA may have a data delay (T+1 or T+2)
- Check if data is available for dates further in the past